In [ ]:
# This cell does not get executed when run with Times Square
import os
import datetime

day_obs = int(os.environ.get("SCHEDVIEW_DAY_OBS", datetime.date.today().strftime("%Y%m%d")))
earliest_sim = datetime.date.fromisoformat(os.environ.get("SCHEDVIEW_SIM_DATE", datetime.date(day_obs//10000, (day_obs%10000)//100, day_obs%100).isoformat()))

In [ ]:
# Validate the input
import re
if not isinstance(earliest_sim, str):
    earliest_sim = earliest_sim.isoformat()

In [ ]:
from IPython.display import display, HTML, Markdown
from collections import OrderedDict
import datetime
import sys
import os
import pandas as pd
import numpy as np
import astropy
import bokeh
import bokeh.io
import boto3
import warnings
import astropy.units as u
from astropy.time import Time, TimezoneInfo
from urllib.parse import urlparse
from lsst.resources import ResourcePath
import yaml
from contextlib import redirect_stdout
import io
from erfa import ErfaWarning

In [ ]:
os.environ['RUBIN_SIM_DATA_DIR'] = '/sdf/data/rubin/shared/rubin_sim_data'

In [ ]:
#sched_source = 'usdf'
sched_source = 'shared'
#sched_source = 'devel'
match sched_source:
    case 'shared':
            sys.path.insert(0, "/sdf/data/rubin/shared/scheduler/packages/rubin_scheduler-3.8.0")
            sys.path.insert(0, "/sdf/data/rubin/shared/scheduler/packages/rubin_sim-2.2.4")
            sys.path.insert(0, "/sdf/data/rubin/shared/scheduler/packages/schedview-0.17.0")
    case 'devel':
        sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/uranography")
        sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/rubin_scheduler")
        sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/rubin_sim")
        sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/schedview")
    case _:
        # Use the current environment
        pass

In [ ]:
import schedview.collect.resources
import schedview.plot
import rubin_sim
from schedview import DayObs

In [ ]:
# Suppress benign warnings.

warnings.filterwarnings(
    "ignore",
    category=ErfaWarning,
    message=r".*ERFA function.*dubious year.*",
)

In [ ]:
archive_uri = "s3://rubin:rubin-scheduler-prenight/opsim/"
compilation_uri = f"{archive_uri}compiled_metadata_cache.h5"

if urlparse(archive_uri).scheme.upper() == 'S3':
    os.environ["LSST_DISABLE_BUCKET_VALIDATION"] = "1"
    os.environ["S3_ENDPOINT_URL"] = "https://s3dfrgw.slac.stanford.edu/"

# Pre-night simulation archive overview with simulations including the `day_obs` of {{ params.day_obs }}

In [ ]:
display(Markdown(f"Including simulations inserted into the archive at `{archive_uri}` on or after {earliest_sim}."))

In [ ]:
day_obs = DayObs.from_date(day_obs)
archives_resource_path = ResourcePath(archive_uri)
num_nights = np.ceil((Time.now() - Time(earliest_sim)).to_value(format='jd')).astype(int)
raw_sim_metadata = rubin_sim.sim_archive.read_archived_sim_metadata(archive_uri, num_nights=num_nights, compilation_resource=compilation_uri)
sim_metadata = schedview.compute.munge_sim_archive_metadata(raw_sim_metadata, day_obs, archive_uri)

### Simonyi telescope

In [ ]:
simonyi_sim_metadata = {
    k: v for k, v in sim_metadata.items()
    if 'telescope' in sim_metadata[k] and sim_metadata[k]['telescope'].lower() == 'simonyi'
}
sim_table_html = schedview.plot.make_html_table_of_sim_archive_metadata(simonyi_sim_metadata, day_obs)
display(HTML(sim_table_html))

### AuxTel

In [ ]:
auxtel_sim_metadata = {
    k: v for k, v in sim_metadata.items()
    if 'telescope' in sim_metadata[k] and sim_metadata[k]['telescope'].lower() == 'auxtel'
}
sim_table_html = schedview.plot.make_html_table_of_sim_archive_metadata(auxtel_sim_metadata, day_obs)
display(HTML(sim_table_html))